In [6]:
from torch import nn
from torch.optim import Adam

In [8]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

ModuleNotFoundError: No module named 'pytorch'